# Reinforcement Learning Agent Simulation for TicTacToe

In [6]:
import random
import math
from matplotlib import pyplot as plt
from statistics import mean 

class TicTacToe:
    def player_input(state, sym):
        chk = 1
        while chk:
            chk=0
            plmove = input("Enter your move (1-9): ")
            if plmove not in [str(i) for i in range(10)]:
                chk = 1
                print("Beep Bop! Invalid Input!")
                continue
            plmove = int(plmove)
            if not TicTacToe.check_move(plmove-1, state):
                chk = 1
                print("Beep Bop! Invalid Input!")
                continue
        state[plmove - 1] = sym
        return state
        
    def pc_input_rand(state, sym):
        chk = 1
        print("'{}' PCs Turn!".format(sym))
        while chk:
            chk=0
            pcmove = random.randrange(9)
            if not TicTacToe.check_move(pcmove, state):
                chk = 1                
        state[pcmove] = sym
        return state
    
    def pc_input_expert(state, sym):
        chk = 1
        print("'{}' PCs Turn!".format(sym))        
        thinkstate = state[:]
        osym = plsym if sym==pcsym else pcsym
        
        #Case 0: Init
        if ("X" not in state) and ("O" not in state):
            state[random.choice([0,2,4,6,8])] = sym
            print("Played Move 0! (Starting Move)")
            return state
        
        #Case 1: winning move
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = sym
                if TicTacToe.check_win(thinkstate, sym):
                    state[i] = sym
                    print("Played Move 1! (Winning Move)")
                    return state
                thinkstate[i] = "_"
                
        #Case 2: block        
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = osym
                if TicTacToe.check_win(thinkstate, osym):
                    state[i] = sym
                    print("Played Move 2! (Block)")
                    return state
                thinkstate[i] = "_"
                
        #Case 3: fork        
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = sym
                winc=0
                for j in range(9):
                    if TicTacToe.check_move(j, thinkstate):
                        thinkstate[j] = sym                        
                        if TicTacToe.check_win(thinkstate, sym):
                            winc+=1
                        if winc >= 2:
                            state[i] = sym
                            print("Played Move 3! (Fork)")
                            return state                    
                        thinkstate[j] = "_"
                thinkstate[i] = "_"
        
        #Case 4: block fork
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = osym
                winc=0
                for j in range(9):
                    if TicTacToe.check_move(j, thinkstate):
                        thinkstate[j] = osym                        
                        if TicTacToe.check_win(thinkstate, osym):
                            winc+=1                                            
                        thinkstate[j] = "_"
                        
                if winc >= 2:
                    for j in range(9):
                        if TicTacToe.check_move(j, thinkstate):
                            thinkstate[j] = sym
                            winc2=0
                            for k in range(9):
                                if TicTacToe.check_move(k, thinkstate):
                                    thinkstate[k] = sym                        
                                    if TicTacToe.check_win(thinkstate, sym):                                        
                                        state[j] = sym
                                        print("Played Move 4! (counter Fork)")
                                        return state                    
                                    thinkstate[k] = "_"
                            thinkstate[j] = "_"   
                    state[i] = sym
                    print("Played Move 4! (block fork)")
                    return state                    
                thinkstate[i] = "_"
        
        #Case 5: centre
        if state[4]=="_":
            state[4] = sym
            print("Played Move 5! (centre)")
            return state
        
        #Case 6: opp corner 
        movs = [0,2,6,8]
        for i in range(4):
            if state[movs[3-i]]=="_":
                if state[movs[i]]==osym:
                    state[movs[3-i]]=sym
                    print("Played Move 6! (opp corner)")
                    return state
        
        #Case 7: empty corner
        for i in random.sample([0,2,6,8],4):
            if state[i]=="_":
                state[i] = sym
                print("Played Move 7! (empty corner)")
                return state
        
        #Case 8: empty side
        for i in random.sample([1,3,5,7],4):
            if state[i]=="_":
                state[i] = sym
                print("Played Move 8! (empty side)")
                return state
        
        print("Confused!")
        return pc_input_rand(state, sym)
    
    def pc_input_search(state, sym):
        chk = 1
        print("'{}' PCs Turn!".format(sym))
        pcmove=state_dfs(state, sym)
        
        state[pcmove] = sym
        return state
    
    def state_dfs(state, sym):        
        for i in range(10):
            if TicTacToe.check_move(i, state):
                state[i] = sym
                state = pc_input_rand(state, plsym)                
                
                if check_status(state, 0) and not check_win(state, sym):
                    #state_dfs(state, sym)
                    pass
    
    def display_board(state):
        for i in range(0,9,3):
            print("{}|{}|{}".format(state[9-i-3],state[9-i-2],state[9-i-1]))
        print("  ")
    
    def check_status(state, disp=1):
        if TicTacToe.check_win(state, plsym):
            if (disp): print("You Won!")
            return 0
        elif TicTacToe.check_win(state, pcsym):
            if (disp): print("You Lost!")
            return 0
        elif "_" not in state:
            if (disp): print("Draw! Game Over!")
            return 0
        else:
            return 1    
    
    def check_move(move, state):
        if state[move] == '_':
            return True
        else:
            return False
        
    def check_win(state, sym):
        winstr = sym+sym+sym
        for i in range(0,9,3):
            if state[i]+state[i+1]+state[i+2] == winstr:
                return True
        for i in range(0,3):
            if state[i]+state[i+3]+state[i+6] == winstr:
                return True
        if (state[0]+state[4]+state[8] == winstr) or (state[2]+state[4]+state[6] == winstr):
            return True
        return False
    

## User Play:- Player vs Random

In [2]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("You won the toss! So start the game first!")
    turn = 1
else :
    print("PC won the toss!")

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.player_input(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_rand(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1
        
TicTacToe.display_board(state)    

Let's Play TicTacToe!
Enter O or X: x
You chose X. Time for a toss!
Press anykey to toss!
You won the toss! So start the game first!
_|_|_
_|_|_
_|_|_
  
Enter your move (1-9): 5
_|_|_
_|X|_
_|_|_
  
'O' PCs Turn!
_|_|O
_|X|_
_|_|_
  
Enter your move (1-9): 7
X|_|O
_|X|_
_|_|_
  
'O' PCs Turn!
X|_|O
_|X|_
O|_|_
  
Enter your move (1-9): 3
You Won!
X|_|O
_|X|_
O|_|X
  


## User Play:- Player vs Strategic

In [10]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("You won the toss! So start the game first!")
    turn = 1
else :
    print("PC won the toss!")

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.player_input(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1
        
TicTacToe.display_board(state)    

Let's Play TicTacToe!
Enter O or X: x
You chose X. Time for a toss!
Press anykey to toss!
You won the toss! So start the game first!
_|_|_
_|_|_
_|_|_
  
Enter your move (1-9): 1
_|_|_
_|_|_
X|_|_
  
'O' PCs Turn!
Played Move 5! (centre)
_|_|_
_|O|_
X|_|_
  
Enter your move (1-9): 9
_|_|X
_|O|_
X|_|_
  
'O' PCs Turn!
Played Move 4! (counter Fork)
_|_|X
_|O|_
X|O|_
  
Enter your move (1-9): 8
_|X|X
_|O|_
X|O|_
  
'O' PCs Turn!
Played Move 2! (Block)
O|X|X
_|O|_
X|O|_
  
Enter your move (1-9): 3
O|X|X
_|O|_
X|O|X
  
'O' PCs Turn!
Played Move 2! (Block)
O|X|X
_|O|O
X|O|X
  
Enter your move (1-9): 4
Draw! Game Over!
O|X|X
X|O|O
X|O|X
  


## Self Play:- Strategic vs Random

In [4]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("Your Agent '{} PC' won the toss! So start the game first!".format(plsym))
    turn = 1
else :
    print("'{}' PC won the toss!".format(pcsym))

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_rand(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1

TicTacToe.display_board(state)    

Let's Play TicTacToe!
Enter O or X: x
You chose X. Time for a toss!
Press anykey to toss!
Your Agent 'X PC' won the toss! So start the game first!
_|_|_
_|_|_
_|_|_
  
'X' PCs Turn!
Played Move 0! (Starting Move)
_|_|_
_|_|_
_|_|X
  
'O' PCs Turn!
_|_|_
_|O|_
_|_|X
  
'X' PCs Turn!
Played Move 7! (empty corner)
_|_|X
_|O|_
_|_|X
  
'O' PCs Turn!
_|_|X
_|O|_
_|O|X
  
'X' PCs Turn!
Played Move 1! (Winning Move)
You Won!
_|_|X
_|O|X
_|O|X
  


## Self Play:- Strategic Vs Strategic

In [5]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("Your Agent '{} PC' won the toss! So start the game first!".format(plsym))
    turn = 1
else :
    print("'{}' PC won the toss!".format(pcsym))

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1

TicTacToe.display_board(state)    

Let's Play TicTacToe!
Enter O or X: x
You chose X. Time for a toss!
Press anykey to toss!
'O' PC won the toss!
_|_|_
_|_|_
_|_|_
  
'O' PCs Turn!
Played Move 0! (Starting Move)
O|_|_
_|_|_
_|_|_
  
'X' PCs Turn!
Played Move 5! (centre)
O|_|_
_|X|_
_|_|_
  
'O' PCs Turn!
Played Move 7! (empty corner)
O|_|O
_|X|_
_|_|_
  
'X' PCs Turn!
Played Move 2! (Block)
O|X|O
_|X|_
_|_|_
  
'O' PCs Turn!
Played Move 2! (Block)
O|X|O
_|X|_
_|O|_
  
'X' PCs Turn!
Played Move 4! (block fork)
O|X|O
_|X|_
X|O|_
  
'O' PCs Turn!
Played Move 7! (empty corner)
O|X|O
_|X|_
X|O|O
  
'X' PCs Turn!
Played Move 2! (Block)
O|X|O
_|X|X
X|O|O
  
'O' PCs Turn!
Played Move 2! (Block)
Draw! Game Over!
O|X|O
O|X|X
X|O|O
  
